# Natural language engineering 2: Assignment

## Classification of Twitter posts regarding the Ukrainian War by emotion

A project by **Johannes Wittmann** and **Quirin Wittmann**.

In [39]:
from pprint import pprint

### Data crawling

### Data preprocessing

### Model finetuning

Neccesary dependencys for finetuning with Huggingface.

In [1]:
!pip install datasets transformers
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 31.8 MB/s 
     |████████████████████████████████| 4.4 MB 1.4 MB/s 
     |████████████████████████████████| 212 kB 75.0 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 1.1 MB 63.9 MB/s 
     |████████████████████████████████| 140 kB 74.4 MB/s 
     |████████████████████████████████| 596 kB 21.7 MB/s 
     |████████████████████████████████| 127 kB 57.1 MB/s 
     |████████████████████████████████| 6.6 MB 47.6 MB/s 
     |████████████████████████████████| 94 kB 5.3 MB/s 
     |████████████████████████████████| 271 kB 76.0 MB/s 
     |████████████████████████████████| 144 kB 76.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing i

Finetuning imports:


In [2]:
from datasets import load_dataset
from transformers import pipeline
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

Load Twitter dataset for emotions: <br>
[https://huggingface.co/datasets/tweet_eval/viewer/emotion/train](https://huggingface.co/datasets/tweet_eval/viewer/emotion/train)

In [3]:
emotion_dataset = load_dataset("tweet_eval", "emotion")
print(emotion_dataset["train"][0])

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emotion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry", 'label': 2}


Embedd tweets:

In [4]:
#checkpoint = "bhadresh-savani/distilbert-base-uncased-emotion"
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Tokenisation function defining:

In [5]:
def tokenize_example(example):
  return tokenizer(example["text"], truncation=True)

Tokenize dataset with function:

In [6]:
emotion_dataset_tokenized = emotion_dataset.map(tokenize_example, batched=True)

Parameter 'function'=<function tokenize_example at 0x7fbe3a525d40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Defining the dataset padder:

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Classification Head:

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Define the evaluation function.
Huggingface reccomends accuracy, recall, precision and F1 for text classification. <br>
[Metrics for Text Classification](https://huggingface.co/tasks/text-classification)

In [9]:
def evaluate(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="macro")
  accuracy = accuracy_score(labels, preds)
  precision = precision_score(labels, preds, average="macro")
  recall = recall_score(labels, preds, average="macro")
  return {"precision": precision, "recall": recall, "accuracy": accuracy, "f1": f1}

Defining the trainingparameters:

In [10]:
batch_size = 8
logging_steps = len(emotion_dataset_tokenized["train"])
model_name = f"{checkpoint}-finetuned-tweet_emotion"
training_args = TrainingArguments(output_dir = model_name, 
                                  num_train_epochs = 2, 
                                  learning_rate = 2e-5, 
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  evaluation_strategy="epoch",
                                  disable_tqdm = False, 
                                  logging_steps = logging_steps, 
                                  log_level="error")

Creating the trainer:

In [11]:
trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = evaluate,
    train_dataset=emotion_dataset_tokenized["train"], 
    eval_dataset=emotion_dataset_tokenized["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Training:

In [12]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Precision,Recall,Accuracy,F1
1,No log,0.564218,0.794232,0.721845,0.802252,0.744573
2,No log,0.565895,0.780857,0.764827,0.811400,0.772001


TrainOutput(global_step=816, training_loss=0.6140071270512599, metrics={'train_runtime': 93.8075, 'train_samples_per_second': 69.44, 'train_steps_per_second': 8.699, 'total_flos': 130324020930792.0, 'train_loss': 0.6140071270512599, 'epoch': 2.0})

In [23]:
trainer.save_model("./outputmodel")

### Classification

In [14]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from transformers import pipeline
from transformers import AutoModelForSequenceClassification

In [27]:
import os
os.listdir("/content/outputmodel/")

['tokenizer.json',
 'training_args.bin',
 'special_tokens_map.json',
 'pytorch_model.bin',
 'config.json',
 'tokenizer_config.json',
 'vocab.txt']

In [36]:
checkpoint = "/content/outputmodel/"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4)
emotion_pipe = pipeline(task="sentiment-analysis", model=model, tokenizer=tokenizer)

print(model.config.id2label)

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3'}
[{'label': 'LABEL_1', 'score': 0.7400911450386047}, {'label': 'LABEL_0', 'score': 0.9874593019485474}, {'label': 'LABEL_1', 'score': 0.9469987750053406}]


In [45]:
# TEST DER PIPELINE
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
    "I love this so much",
    "I am looking forward to it",
    "We must stay optimistic",
    "Why did this happen?",
    "Why did you do this to me?",
    "Why didn't you clean this mess?",
    "I am sorry",
]
#0: anger
#1: joy
#2: optimism
#3: sadness
pprint(emotion_pipe(raw_inputs))

[{'label': 'LABEL_1', 'score': 0.7400911450386047},
 {'label': 'LABEL_0', 'score': 0.9874593019485474},
 {'label': 'LABEL_1', 'score': 0.9469987750053406},
 {'label': 'LABEL_1', 'score': 0.9313478469848633},
 {'label': 'LABEL_2', 'score': 0.9061725735664368},
 {'label': 'LABEL_3', 'score': 0.742118775844574},
 {'label': 'LABEL_0', 'score': 0.9222608208656311},
 {'label': 'LABEL_0', 'score': 0.9551247954368591},
 {'label': 'LABEL_3', 'score': 0.9514026045799255}]


In [ ]:
#predictions = emotion_pipe( HIER GECRAWLTE TWEET-LISTE)

In [ ]:
predicted_labels = []
for prediction in predictions:
  predicted_labels.append(prediction['label'].lower())

### Data analysis and postprocessing

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fa2802eb-518b-4254-91a1-6660d89efb87' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>